In [40]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import matplotlib.pyplot as plt
import numpy as np

# load cleaned data
data = pd.read_csv('data/00_processed.csv')
data.head()

,YEAR,ALUMNI_NATIONAL,ALUMNI_AFRICA,ALUMNI_ASIA,ALUMNI_CENTRAL_AMERICA_AND_CARIBBEAN,ALUMNI_EUROPEAN_UNION,ALUMNI_NORTH_AMERICA,ALUMNI_REST_OF_EUROPE,ALUMNI_SOUTH_AMERICA,ALUMNI_LAIN_LAIN,...,CITATION_500_1000,CITATION__1000_3000,CITATION_3000_5000,CITATION_5000_10000,MENTION_NEGATIVE,MENTION_NEUTRAL,MENTION_POSITIVE,CONSULTATION_CONFERENCE_SEMINAR,CONSULTATION_CONSULTANCY,CONSULTATION_TRAINING_SHORTCOURSES
0,2020,4925,50,99,1.0,1.000000,4.0,3.0,1.0,16,...,44,20,4,3,1.000000,45.00,33.0,12.00,15.0,4.00
1,2021,6942,226,589,1.0,2.000000,1.0,2.0,1.0,6,...,45,20,4,3,2.000000,42.00,59.0,22.00,10.0,5.00
2,2022,8659,275,763,1.0,1.333333,2.5,4.0,1.0,21,...,44,20,4,3,1.333333,2.00,24.0,15.00,17.0,10.00
3,2023,7289,257,849,1.0,1.333333,1.0,2.0,1.0,11,...,42,20,4,3,1.333333,22.75,1.0,20.00,16.0,20.00
4,2024,6815,265,828,1.0,1.000000,4.0,4.0,1.0,8,...,41,19,4,3,1.000000,2.00,3.0,17.25,14.5,9.75


## PCA

In [44]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Separate features and target
X = data.drop('SCORE_AR', axis=1)
y = data['SCORE_AR']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=0.95)  # 95% of variance retained
X_pca = pca.fit_transform(X_scaled)

# Check how many components were kept
print("Number of components selected:", X_pca.shape[1])

# Create a DataFrame with the principal components
df_pca = pd.DataFrame(data=X_pca, columns=[f"PC{i}" for i in range(1, X_pca.shape[1] + 1)])
df_pca.head()


Number of components selected: 4


,PC1,PC2,PC3,PC4
0,-5.509423,-5.638592,1.346301,2.653608
1,-4.100439,0.762382,-3.737240,-4.198922
2,-1.211353,5.172736,4.734327,-0.900230
3,2.704400,3.559674,-3.112991,4.348908
4,8.116814,-3.856200,0.769604,-1.903365


## Feature Importance

In [43]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Calculate correlation with the target
correlations = data.corr()['SCORE_AR'].sort_values(ascending=False)
print("Top features by correlation:\n", correlations)

# Use Random Forest for feature importance
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)
importances = model.feature_importances_

# Sort features by importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("\n\nTop features by importance:\n", feature_importance.head(10))


Top features by correlation:
 SCORE_AR                                                1.000000
YEAR                                                    0.996456
PROJECT_AREAS_SUSTAINABLE_AND_RESILIENT_URBANISATION    0.982467
PROMINENT_H_INDEX___50                                  0.940507
PROMINENT_H_INDEX_30_50                                 0.940507
                                                          ...   
PARTNERSHIP_NORTH_AMERICA                                    NaN
PARTNERSHIP_OCEANIA                                          NaN
CONFERENCE_SUMMER_SCHOOL                                     NaN
CITATION_3000_5000                                           NaN
CITATION_5000_10000                                          NaN
Name: SCORE_AR, Length: 71, dtype: float64


Top features by importance:
                                               feature  importance
66                                   MENTION_POSITIVE    0.057887
60                                  CITATION_500_

## Training

In [47]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

X_reduced = df_pca

# Assuming `X_reduced` contains the selected features or PCA components and `y` is your target
model = LinearRegression()
model.fit(X_reduced, y)

# Evaluate using cross-validation (if possible with limited data)
scores = cross_val_score(model, X_reduced, y, cv=3)  # Adjust cv based on data availability
print("Cross-validation scores:", scores)
print("Mean score:", scores.mean())


Cross-validation scores: [-11.39757484  -0.7864857           nan]
Mean score: nan


c:\Users\sharu\.conda\envs\pandasai\lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [ ]:


# Check for stationarity
def test_stationarity(timeseries):
    # Determing rolling statistics
    rolmean = timeseries.rolling(window=12).mean()
    rolstd = timeseries.rolling(window=12).std()

    # Plot rolling statistics:
    import matplotlib.pyplot as plt
    plt.plot(timeseries, color='blue', label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)

test_stationarity(data['SCORE_AR'])


In [ ]:



# bootstrap sample with z-score normalization
def bootstrap_features(data, target_col='SCORE_AR', n_samples=1000):
    # Separate features and target
    features = data.drop(columns=[target_col])
    target = data[target_col]
    
    # Bootstrap features
    feature_indices = resample(
        np.arange(len(features)), 
        n_samples=n_samples,
        replace=True,
        random_state=42
    )
    
    # Create bootstrapped dataset
    bootstrapped_features = features.iloc[feature_indices]
    bootstrapped_target = target.iloc[feature_indices]
    
    # Combine features and target
    bootstrapped_data = pd.concat([bootstrapped_features, bootstrapped_target], axis=1)
    
    return bootstrapped_data

# Apply bootstrapping
bootstrapped_data = bootstrap_features(data, target_col='SCORE_AR', n_samples=1000)

# Save bootstrapped data
bootstrapped_data.to_csv('bootstrapped_data.csv', index=False)

# calculate the mean of the top features
top_feature_means = bootstrapped_data[top_features].mean()

# display the top features and their mean values in dataframe
top_feature_means_df = pd.DataFrame(top_feature_means, columns=['Mean'])
top_feature_means_df

In [11]:
# Training a model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Continue with model training using bootstrapped data
X = bootstrapped_data[top_features]
y = bootstrapped_data['SCORE_AR']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict and calculate mean squared error
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1.4108619517696752e-27


In [12]:
# Increase each feature in the dataset by 20%
X_increased = X * 1.2

# Predict the SCORE_AR for the modified dataset
y_pred_increased = model.predict(X_increased)

# Display the predictions
print(y_pred_increased)

[46.31  47.767 44.954 47.767 47.767 43.443 44.954 44.954 44.954 47.767
 46.31  44.954 47.767 43.443 46.31  43.443 46.31  47.767 39.4   46.31
 43.443 47.767 46.31  39.4   39.4   44.954 44.954 43.443 46.31  46.31
 44.954 46.31  46.31  39.4   44.954 47.767 44.954 47.767 39.4   43.443
 46.31  39.4   46.31  43.443 43.443 39.4   43.443 47.767 43.443 46.31
 46.31  46.31  46.31  47.767 44.954 39.4   46.31  43.443 46.31  43.443
 43.443 46.31  47.767 43.443 43.443 46.31  43.443 43.443 46.31  46.31
 39.4   47.767 47.767 43.443 47.767 43.443 39.4   46.31  46.31  46.31
 47.767 39.4   47.767 47.767 39.4   39.4   39.4   39.4   46.31  44.954
 44.954 39.4   44.954 44.954 39.4   44.954 47.767 43.443 43.443 39.4
 46.31  39.4   46.31  43.443 39.4   47.767 44.954 46.31  44.954 44.954
 39.4   44.954 47.767 44.954 39.4   47.767 43.443 44.954 39.4   43.443
 43.443 46.31  47.767 44.954 39.4   46.31  47.767 46.31  47.767 47.767
 44.954 47.767 46.31  47.767 44.954 44.954 46.31  43.443 43.443 47.767
 39.4   47.76

In [24]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd

def predict_inverse(model, desired_score, top_features, feature_bounds=None, max_iter=2000):
    """
    Predict feature values for desired score using optimization
    """
    initial_guess = bootstrapped_data[top_features].mean().values
    
    if feature_bounds is None:
        feature_bounds = []
        for feature in top_features:
            min_val = bootstrapped_data[feature].min()
            max_val = bootstrapped_data[feature].max() * 100
            feature_bounds.append((min_val, max_val))
    
    def objective(x):
        x_df = pd.DataFrame([x], columns=top_features)
        pred = model.predict(x_df)[0]
        return abs(pred - desired_score)  # Changed to absolute difference
    
    best_result = None
    best_score = float('inf')
    
    # Try multiple optimization methods
    methods = ['L-BFGS-B', 'SLSQP', 'Nelder-Mead']
    
    # Try more starting points
    multipliers = [0.5, 1.0, 1.5, 2.0, 5.0, 10.0]
    starting_points = [initial_guess * m for m in multipliers]
    
    for method in methods:
        for start in starting_points:
            try:
                result = minimize(
                    objective,
                    start,
                    bounds=feature_bounds if method != 'Nelder-Mead' else None,
                    method=method,
                    options={
                        'maxiter': max_iter,
                        'ftol': 1e-8,
                        'gtol': 1e-8
                    }
                )
                
                if result.fun < best_score:
                    best_score = result.fun
                    best_result = result
                    
                # Early stop if we're close enough
                if best_score < 0.1:
                    break
                    
            except Exception as e:
                print(f"Method {method} failed: {str(e)}")
                continue
    
    optimized_values = pd.DataFrame([best_result.x], columns=top_features)
    predicted = model.predict(optimized_values)[0]
    
    print(f"Best optimization method achieved error: {best_score:.4f}")
    print(f"Optimization status: {best_result.message}")
    
    return optimized_values, predicted

# Example usage
desired_score = 50
feature_values, predicted_score = predict_inverse(model, desired_score, top_features)

print(f"Desired Score: {desired_score}")
print(f"Achieved Score: {predicted_score:.2f}")
print(f"Optimization Success: {abs(desired_score - predicted_score) < 1}")
display(feature_values)

C:\Users\sharu\AppData\Local\Temp\ipykernel_20024\3146108622.py:36: OptimizeWarning: Unknown solver options: gtol
  result = minimize(
C:\Users\sharu\AppData\Local\Temp\ipykernel_20024\3146108622.py:36: OptimizeWarning: Unknown solver options: gtol
  result = minimize(
C:\Users\sharu\AppData\Local\Temp\ipykernel_20024\3146108622.py:36: OptimizeWarning: Unknown solver options: gtol
  result = minimize(
C:\Users\sharu\AppData\Local\Temp\ipykernel_20024\3146108622.py:36: OptimizeWarning: Unknown solver options: gtol
  result = minimize(
C:\Users\sharu\AppData\Local\Temp\ipykernel_20024\3146108622.py:36: OptimizeWarning: Unknown solver options: gtol
  result = minimize(
C:\Users\sharu\AppData\Local\Temp\ipykernel_20024\3146108622.py:36: OptimizeWarning: Unknown solver options: gtol
  result = minimize(
C:\Users\sharu\AppData\Local\Temp\ipykernel_20024\3146108622.py:36: OptimizeWarning: Unknown solver options: ftol, gtol
  result = minimize(
C:\Users\sharu\AppData\Local\Temp\ipykernel_20024

Best optimization method achieved error: 3.0810
Optimization status: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Desired Score: 50
Achieved Score: 46.92
Optimization Success: False


Feature,PROJECT_AREAS_ENERGY_SUSTAINABILITY,KEYNOTE_SPEAKER_INVITED,PARTNERSHIP_COUNTRY_N_A,PARTNERSHIP_NATIONAL,PROJECT_AREAS_BIOMEDICAL_AND_HEALTHCARE_ENGINEERING,PROJECT_AREAS_ENERGY_SECURITY,PROJECT_AREAS_NATURAL_PRODUCTS_BIOREFINERY_AND_BIOTECHNOLOGY,PROJECT_AREAS_SMART_LIVING_AND_SUSTAINABLE_CITIES,PROJECT_AREAS_SMART_MANUFACTURING_AND_MATERIALS,PROJECT_AREAS_SUSTAINABLE_AND_RESILIENT_URBANISATION,SCHOLARSHIP_PHD,SCHOLARSHIP_MASTER,CONFERENCE_NATIONAL_INCLUDE_UNIVERSITY_LEVEL,PAPER_INCENTIVE_NO_OF_PAPER_STAFF,SCORE_OVERALL_RANK
0,43.419,329.1795,119.931,73.341,49.035,53.562,66.957,106.521,93.291,110.268,109.38,28.77,66.996,1008.42,284.673
